In [1]:
import pathlib
import numpy as np
import pandas as pd

from utils import *
from viz import *
from task_system import TaskSystem
from policies.edf import EDF
from policies.fjp import FJP

import plotly.express as px
import plotly.graph_objects as go

import os
os.environ["IMAGEIO_FFMPEG_EXE"] = '/Users/abdulhakeemabdulrahman/homebrew/bin/ffmpeg'

import  moviepy.editor as mpy
import io 
from PIL import Image

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
def run(filename, out_filename, policy, atomic=False, steps=None):
    sys = TaskSystem()
    sys.load_tasks(filename=filename)

    if steps is None:
        steps = 4 * sys.L

    lines = []
    for t in range(1, steps+1):
        index = policy(sys.tasks, atomic=atomic)
        # print(t, index)
        sys.step(index)

        n_repeats = t // sys.L
        if (t % sys.L == 0) and (n_repeats > 0):
            start = (n_repeats - 1) * sys.L
            line = sys.trace[start: (start + sys.L)]
            
            if n_repeats > 1 and line == lines[0]:            
                break

            lines.append(line)
        
    with open(out_filename, 'w') as fp:
        for line in lines:
            line = [str(i) for i in line]
            line = ' '.join(line)
            fp.writelines(line + '\n')

    return out_filename
        

In [4]:
filename_set = f'data/set_edf.txt'

name = 'edf_fifo_non_preemptive_edf_set'
filename_trace = f'data/trace_{name}.txt'

output_dir = f'outputs/{name}'
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

In [5]:
# priorities = [1, 2, 3, 4]
# policy = FJP(priorities)

policy = EDF(fifo=True)

In [6]:
run(filename_set, filename_trace, 
    policy=policy, atomic=False, steps=None)

'data/trace_edf_fifo_non_preemptive_edf_set.txt'

In [7]:
tasks = read_tasks(filename_set)
traces = read_traces(filename_trace)
df_traces = process_traces(traces)
# df_traces

In [8]:
plot_polar(tasks, df_traces, 600, output_dir=output_dir)

In [9]:
clip = mpy.ImageSequenceClip(output_dir, fps=1)
clip.write_gif(f'outputs/{name}.gif')

MoviePy - Building file outputs/edf_fifo_non_preemptive_edf_set.gif with imageio.


In [10]:
output_filename = f'outputs/gantt_{name}.png'
plot_gantt(tasks, df_traces, output_filename=output_filename)